<a href="https://colab.research.google.com/github/wynniegross1/vegetation_anomalies/blob/main/vci_calculation_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Calculating VCI for a Specified Region of Interest**


---


Original GEE Script Author: Aaron Eubank

Python Script Adaptation: Wynnie Gross


---


*   This script is run primarily to get a map of Vegetation Condition Index values.
* This script defines a function to calculate this for a given region of interest.   
* The default dataset is MODIS 8-day product, however it is parameterizable to choose a different dataset.
*   Methodology adapted from GEARS - Geospatial Ecology and Remote Sensing - https://www.geospatialecology.com

In [ ]:
import geemap
import ee
from google.colab import userdata
from datetime import datetime

# INITIALIZE GEE PROJECT #
ee.Authenticate()
ee.Initialize(project=userdata.get('projectname'))

def calculate_vci(
    region_of_interest,
    start_date,
    end_date,
    data_source='MODIS', # User can specify MODIS, Landsat, or Sentinel-2
    mask_cropland=True,
    cloud_masking=True,
    month_range=None,
    day_range=None,
    export_to_drive=False, # Trigger export if True
    export_description='VCI_Export',
    export_folder='VCI_Exports',
    export_scale=None
):
    """
    Calculates Vegetation Condition Index (VCI) for a given region and time period.

    Args:
        region_of_interest: ee.FeatureCollection. The area to analyze.
        start_date (str): Start date for the period (YYYY-MM-DD).
        end_date (str): End date for the period (YYYY-MM-DD).
        data_source (str): 'MODIS', 'Landsat', or 'Sentinel-2'.
        mask_cropland (bool): Whether to apply a cropland mask. Default True.
        cloud_masking (bool): Whether to apply cloud masking. Default True.
        month_range (list, optional): List of months to include (1-12) within the start and end date range (i.e. only include growing season). ([X,X])
        day_range (list, optional): List of days of the year to include (1-366) within start and end date range (i.e. only include growing season). ([X,X])
        export_to_drive (bool): Whether to export the result to Google Drive. - Default is False, set to True to export.
        export_description (str): If export_to_drive is True, Name of the export task.
        export_folder (str): If export_to_drive is True, Google Drive folder for export.
        export_scale (int): If export_to_drive is True, Pixel resolution for export.

    Returns:
        tuple: (ee.Image, geemap.Map) The VCI image and the map object.
    """
    # Define data source parameters
    data_sources = {
        'MODIS': {
            'collection': 'MODIS/061/MOD09A1',
            'nir_band': 'sur_refl_b02',
            'red_band': 'sur_refl_b01',
            'cloud_band': 'StateQA',
            'cloud_mask_bit': 1 << 10,
            'scale': 500
        },
        'Landsat': {
            'collection': 'LANDSAT/LC08/C02/T1_L2',
            'nir_band': 'SR_B5',
            'red_band': 'SR_B4',
            'cloud_band': 'QA_PIXEL',
            'cloud_mask_bit': 1 << 3,
            'scale': 30
        },
        'Sentinel-2': {
            'collection': 'COPERNICUS/S2_SR_HARMONIZED',
            'nir_band': 'B8',
            'red_band': 'B4',
            'cloud_band': 'QA60',
            'cloud_mask_bit': 1 << 10,
            'scale': 10
        }
    }

    source = data_sources[data_source]

    # Helper functions
    def mask_clouds(image):
        """Masks clouds using the appropriate band for each data source."""
        QA = image.select([source['cloud_band']])
        return image.updateMask(QA.bitwiseAnd(source['cloud_mask_bit']).eq(0))

    def crop_mask(image):
        """Masks non-cropland areas using ESA WorldCover."""
        esa = ee.ImageCollection('ESA/WorldCover/v100')
        esa_latest = esa.limit(1, 'system:time_start').first()
        cropland = esa_latest.updateMask(
            esa_latest.select('Map').eq(40).clip(region_of_interest)
        )
        return image.updateMask(cropland)

    def add_ndvi(image):
        """Calculates NDVI and adds it as a band."""
        ndvi = image.normalizedDifference([source['nir_band'], source['red_band']]).rename('ndvi')
        return image.addBands(ndvi)

    # Load the image collection
    collection = ee.ImageCollection(source['collection'])

    # Filter by region of interest and date(s)
    collection = ee.ImageCollection(source['collection']).filterBounds(region_of_interest)

    # Temporal filtering for either month range or day of year range
    temporal_filter = ee.Filter.date(start_date, end_date)

    if month_range:
        temporal_filter = temporal_filter.And(
            ee.Filter.calendarRange(month_range[0], month_range[1], 'month')
        )
    if day_range:
        temporal_filter = temporal_filter.And(
            ee.Filter.calendarRange(day_range[0], day_range[1], 'day_of_year')
        )

    collection = collection.filter(temporal_filter)  # Updated date filtering

    # Apply cloud masking if requested
    if cloud_masking:
        collection = collection.map(mask_clouds)

    # Calculate NDVI
    ndvi_collection = collection.map(add_ndvi).select('ndvi')

    # Apply cropland mask if requested
    if mask_cropland:
        ndvi_collection = ndvi_collection.map(crop_mask)

    # Calculate min and max NDVI
    ndvi_min = ndvi_collection.min()
    ndvi_max = ndvi_collection.max()

    # Calculate VCI
    vci = ndvi_collection.map(lambda img:
        img.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).multiply(100)
    ).mean()

    # Create a map object
    Map = geemap.Map(center=region_of_interest.geometry().centroid().getInfo()['coordinates'][::-1], zoom=7)

    # Visualization parameters
    vci_vis = {
        'min': 0,
        'max': 100,
        'palette': ['red', 'yellow', 'green']
    }

    # Add the VCI layer to the map
    Map.addLayer(vci, vci_vis, 'Vegetation Condition Index')

    # Adding Outline to the Map
    outline = region_of_interest.style(fillColor='00000000')
    Map.addLayer(outline, {}, "Region of Interest")

    # Add legend
    Map.add_colorbar(
        vci_vis, label="Vegetation Condition Index",
        layer_name="VCI", orientation="horizontal"
    )

    # Optional: Export to Google Drive
    if export_to_drive:
      source_scales = {'MODIS': 500, 'Landsat': 30, 'Sentinel-2': 10} # Example scales
      scale = export_scale if export_scale else source_scales.get(source, 10) # Default to 10 if source not found

      region_geom = region_of_interest.geometry().bounds()
      task = ee.batch.Export.image.toDrive(
          image=vci,
          description=export_description,
          folder=export_folder,
          scale=scale,
          region=region_geom,
          maxPixels=1e13,
          fileFormat='GeoTIFF'
      )
      task.start()
      print("Export started. Monitoring status...")

      while task.active():
          print('Current status:', task.status()['state'])
          time.sleep(30)  # Check every 30 seconds

      print('Final status:', task.status())

    return vci, Map

# Function to define region of interest based on country, admin1, and/or admin2
def get_region_of_interest(country, admin1_list=None, admin2_list=None):
    """
    Filters the admin2 boundaries dataset by country, and optionally by multiple admin1 and/or admin2 regions.

    Args:
        country (str): The name of the country (field 'admin0').
        admin1_list (list, optional): List of admin1 regions (field 'admin1').
        admin2_list (list, optional): List of admin2 regions (field 'admin2').

    Returns:
        ee.FeatureCollection: Filtered region of interest.

    Raises:
        ValueError: If no region matches the provided filters.
    """
    # Load the world admin2 boundaries dataset
    admin2_dataset = ee.FeatureCollection("projects/ee-aeubank/assets/world_admin2")

    # Start with a filter for the country
    filters = [ee.Filter.eq('admin0', country)]

    # Add filters for multiple admin1 regions if provided
    if admin1_list:
        admin1_filters = [ee.Filter.eq('admin1', region) for region in admin1_list]
        filters.append(ee.Filter.Or(*admin1_filters))

    # Add filters for multiple admin2 regions if provided
    if admin2_list:
        admin2_filters = [ee.Filter.eq('admin2', region) for region in admin2_list]
        filters.append(ee.Filter.Or(*admin2_filters))

    # Combine all filters using an AND filter
    combined_filter = ee.Filter.And(*filters)

    # Apply the combined filter to the dataset
    region_of_interest = admin2_dataset.filter(combined_filter)

    # Check if the filtered collection is empty
    count = region_of_interest.size().getInfo()
    if count == 0:
        raise ValueError(
            f"No matching region found for country='{country}', "
            f"admin1_list={admin1_list}, admin2_list={admin2_list}. "
            "Please check your input values."
        )

    return region_of_interest

# Example usage
# Calculate VCI and get map
vci_result, vci_map = calculate_vci(
    region_of_interest = get_region_of_interest('Sudan', ['Sennar','Aj Jazirah']),
    start_date='2023-01-01',
    end_date='2023-12-31',
    data_source='Sentinel-2',  # Can be 'MODIS', 'Landsat', or 'Sentinel-2'
    day_range= [91,244],
    export_to_drive=False
)

# Display the map
vci_map